Will create a few datasets of varying complexity

First try two prisons, with one court for each prison.

Prison 200km apart.

Courts 50km from prison

In [37]:
# This is what the target data looks like
import pandas as pd
pd.read_csv("/Users/robinlinacre/Documents/visualisation_projects/discrete_prison_location_calculator/data/test_data_2.csv", nrows=2)

,source_court,source_postcode,dest_postcode,source_lat,source_lng,demand,dest_prison,dest_lat,dest_lng,distance,date,prison_capacity,duration
0,Aberdare,CF44 0NW,BL2 6LU,51.714098,-3.439116,0.890747,0,53.577688,-2.38176,219.305171,2016-01-01,20,219.305171
1,Abergavenny,NP7 5DL,BL2 6LU,51.821759,-3.023196,0.393398,0,53.577688,-2.38176,200.083983,2016-01-01,20,200.083983


In [38]:
import psycopg2

conn = "host='localhost' dbname='postgres' user='postgres' password=''"
conn = psycopg2.connect(conn)

In [39]:
# Now for each of the prisons, generate n courts 

def generate_point_dict(start_point,  bearing_degrees, distance_km,):
    sql = """
        select ST_x(ST_Project(p.point, {distance_meters}, radians({degrees}))::geometry) as lng,
        ST_y(ST_Project(p.point, {distance_meters}, radians({degrees}))::geometry) as lat 
        
        from  (select ST_SetSRID(ST_MakePoint({lng},{lat}),4326) as point) as p
    """
    
    this_sql = sql.format(lat=start_point["lat"], lng=start_point["lng"], degrees=bearing_degrees, distance_meters=distance_km*1000)
    return pd.read_sql(this_sql, conn).loc[0].to_dict()

In [40]:
import pandas as pd
import random
prisons = []



num_prisons = 2
distances_km = [300]
bearings_degrees = [0]
sizes = [800,1200]

start_prison = {
    "lng": -0.142375,
    "lat": 51.5011093,
    "prison_capacity": 500
}
prisons.append(start_prison)

for i in range(num_prisons-1):
    distance_km = 300
    bearing_degrees = 0
    this_prison = generate_point_dict(start_prison, bearing_degrees, distance_km)
    this_prison["prison_capacity"] = 500
    prisons.append(this_prison)
prisons_df = pd.DataFrame(prisons).reset_index()
prisons_df.columns=["dest_prison", "dest_lat", "dest_lng", "prison_capacity"]
prisons_df

,dest_prison,dest_lat,dest_lng,prison_capacity
0,0,51.501109,-0.142375,500
1,1,54.196930,-0.142375,500


In [41]:
courts = []

In [42]:
prison_dict = prisons_df.loc[0].to_dict()
this_prison = {"lat":prison_dict["dest_lat"], "lng": prison_dict["dest_lng"]}
distance_km = 50
bearing_degrees = 0
this_court = generate_point_dict(this_prison, bearing_degrees, distance_km)
this_court["demand"] = 400
courts.append(this_court)

In [43]:
prison_dict = prisons_df.loc[1].to_dict()
this_prison = {"lat":prison_dict["dest_lat"], "lng": prison_dict["dest_lng"]}
distance_km = 50
bearing_degrees = 180
this_court = generate_point_dict(this_prison, bearing_degrees, distance_km)
this_court["demand"] = 600
courts.append(this_court)

In [44]:
        
courts_df = pd.DataFrame(courts).reset_index()
courts_df.columns=["source_court", "demand", "source_lat", "source_lng"]


In [45]:
# Cartestian product
prisons_df["cart"] =1
courts_df["cart"] = 1
all_combos = prisons_df.merge(courts_df)
all_combos = all_combos.drop("cart", axis=1)
all_combos

,dest_prison,dest_lat,dest_lng,prison_capacity,source_court,demand,source_lat,source_lng
0,0,51.501109,-0.142375,500,0,400,51.950498,-0.142375
1,0,51.501109,-0.142375,500,1,600,53.747712,-0.142375
2,1,54.196930,-0.142375,500,0,400,51.950498,-0.142375
3,1,54.196930,-0.142375,500,1,600,53.747712,-0.142375


In [46]:
# Add distance date and duration
sql = """
    select ST_Distance(
    ST_Transform(ST_SetSRID(ST_MakePoint({lng1},{lat1}),4326),27700),
    ST_Transform(ST_SetSRID(ST_MakePoint({lng2},{lat2}),4326),27700)
    )/1000 as distance;
"""

for r in all_combos.iterrows():
    index = r[0]
    row = r[1]
    
    lat1 = row["dest_lat"]
    lng1 = row["dest_lng"]
    
    lat2= row["source_lat"]
    lng2 = row["source_lng"]
    
    df_distance=pd.read_sql(sql.format(lat1 = lat1, lat2=lat2, lng1=lng1, lng2=lng2 ),conn)
    distance = df_distance.loc[0,"distance"]
    
    all_combos.loc[index,"distance"] = distance
    all_combos.loc[index,"duration"] = distance*2

all_combos["date"] = "2016-01-01"


In [47]:
path = "/Users/robinlinacre/Documents/visualisation_projects/discrete_prison_location_calculator/data/qa_dataset_1.csv"
all_combos.to_csv(path, index=False)

In [48]:
all_combos

,dest_prison,dest_lat,dest_lng,prison_capacity,source_court,demand,source_lat,source_lng,distance,duration,date
0,0,51.501109,-0.142375,500,0,400,51.950498,-0.142375,49.991574,99.983148,2016-01-01
1,0,51.501109,-0.142375,500,1,600,53.747712,-0.142375,249.955892,499.911783,2016-01-01
2,1,54.196930,-0.142375,500,0,400,51.950498,-0.142375,249.954909,499.909818,2016-01-01
3,1,54.196930,-0.142375,500,1,600,53.747712,-0.142375,49.990591,99.981182,2016-01-01
